# Multi Agent Demo Recursive and Langraph Combined

See link [LANGGRAPH_COMPARSION](https://github.com/hankbesser/recursive-companion-complete/blob/main/docs/LANGGRAPH_COMPARISON.md) for more.

In [ ]:
# SPDX-License-Identifier: MIT
#
# Copyright (c) [2025] [Henry Besser]
#
# This software is licensed under the MIT License.
# See the LICENSE file in the project root for the full license text.

# demos/langgraph.ipynb

In [ ]:
#from dotenv import load_dotenv
#load_dotenv()

In [ ]:
import os
api_key_status = "Loaded" if os.getenv("OPENAI_API_KEY") else "NOT FOUND - Check your .env file and environment."
print(f"OpenAI API Key status: {api_key_status}")

In [ ]:
# imports
from IPython.display import Image, display, Markdown
from langchain_core.runnables import RunnableLambda
from langgraph.graph import StateGraph
from typing import TypedDict
from recursive_companion.base import MarketingCompanion, BugTriageCompanion, StrategyCompanion

In [ ]:
# different models for differents domains
llm_fast  = "gpt-4o-mini"
llm_deep  = "gpt-4.1-mini" 

In [ ]:
# create the agents
# tip read the doctring by hovering over the class
mkt   = MarketingCompanion(llm=llm_fast, temperature=0.8, max_loops=3, similarity_threshold=0.96)
eng   = BugTriageCompanion(llm=llm_deep, temperature=0.3)
plan = StrategyCompanion(llm=llm_fast)

In [ ]:
# Each node is now a first-class Runnable; you get built-in tracing, concurrency, retries, etc., without rewriting your engine.
mkt_node  = RunnableLambda(mkt)          # __call__ alias does the trick
eng_node  = RunnableLambda(eng)

In [ ]:
# merge-lambda joins text views into one string
# note: LangGraph passes the entire upstream-state dict to a node.
# with out this function, two upstream nodes are piped straight into strategy, 
# so plan_node will receive a Python dict like {"engineering": "...", "marketing": "..."}.
# That's fine if your StrategyCompanion prompt expects that JSON blob, 
# but most of the time you'll want to concatenate the two strings first.

merge_node = RunnableLambda(
    lambda d: f"### Marketing\n{d['marketing']}\n\n### Engineering\n{d['engineering']}"
)
plan_node  = RunnableLambda(plan)

# Define the state schema for LangGraph
class GraphState(TypedDict):
    input: str
    marketing: str
    engineering: str
    merged: str
    final_plan: str

# Inline LangGraph example (fan-in)
# No extra prompts, no schema gymnastics: simply passing text between the callables the classes already expose.
graph = StateGraph(GraphState)
graph.add_node("marketing_agent",    lambda state: {"marketing": mkt_node.invoke(state["input"])})
graph.add_node("engineering_agent",  lambda state: {"engineering": eng_node.invoke(state["input"])})
graph.add_node("merge_agent",        lambda state: {"merged": merge_node.invoke(state)})
graph.add_node("strategy_agent",     lambda state: {"final_plan": plan_node.invoke(state["merged"])})

graph.add_edge("marketing_agent", "merge_agent")
graph.add_edge("engineering_agent", "merge_agent")
graph.add_edge("merge_agent", "strategy_agent")

graph.add_edge("__start__", "marketing_agent")
graph.add_edge("__start__", "engineering_agent")
graph.set_finish_point("strategy_agent")
workflow = graph.compile()

In [ ]:
#display the graph
display(Image(workflow.get_graph().draw_mermaid_png()))

In [ ]:
result = workflow.invoke(
    {"input": "App ratings fell to 3.2★ and uploads crash on iOS 17.2. Diagnose & propose next steps."}
)

In [ ]:
final = result.get("final_plan", "")

In [ ]:
print("\n=== FINAL PLAN ===\n")
display(Markdown(final))

In [ ]:
# === After LangGraph workflow completes ===
print("\n🔍 DEEP INTROSPECTION - What LangGraph CAN'T normally show you:\n")
# Show iteration counts
print(f"Marketing iterations: {len(mkt.run_log)}")
print(f"Engineering iterations: {len(eng.run_log)}")
print(f"Strategy iterations: {len(plan.run_log)}")
# Show why each converged
print("\n📊 CONVERGENCE ANALYSIS:")
for name, agent in [("Marketing", mkt), ("Engineering", eng), ("Strategy", plan)]:
    if len(agent.run_log) < agent.max_loops:
        print(f"{name}: Converged early (quality threshold reached)")
    else:
        print(f"{name}: Used all {agent.max_loops} iterations")


In [ ]:
# Show full thinking process for marketing agent
print("\n🧠 MARKETING THINKING PROCESS:")
display(Markdown(mkt.transcript_as_markdown()))


In [ ]:
# Show full thinking process for engineering agent
print("\n🔧 ENGINEERING THINKING PROCESS:")
display(Markdown(eng.transcript_as_markdown()))


In [ ]:
# Show full thinking process for strategy agent after merged view
print("\n🎯 STRATEGY SYNTHESIS PROCESS:")
display(Markdown(plan.transcript_as_markdown()))